## DATA PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
df = pd.read_excel("category1.xlsx",sheet_name= [0,1,2,3,4,5,6,7])

In [3]:
allUserData = df[3]
userData = [df[3], df[4], df[5], df[6], df[7]]

for i in userData[1:]:
    allUserData = allUserData.append(i)

In [4]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','location_name','location_category','my_place','location_id','place_id'], 1)

In [5]:
def tomins(ts):
    a = ts.hour*60 + ts.minute
    return a

allUserData['start_time'] = allUserData['start_time'].apply(tomins)
allUserData['end_time'] = allUserData['end_time'].apply(tomins)
allUserData['duration'] = allUserData['duration'].apply(tomins)

allUserData.tail()

,user_id,weekday,start_time,end_time,duration,category,category_id,location
69,5,5,0,45,45,home,2,1
70,5,5,45,470,425,sleep,3,1
71,5,5,470,490,20,food,1,1
72,5,5,490,565,75,commute,17,99
73,5,5,565,810,245,work,6,3


In [6]:
cols = allUserData.columns
for col in cols:
    print(col)

user_id
weekday
start_time
end_time
duration
category
category_id
location


## Random Forest Model

In [7]:
train_df = allUserData[allUserData['weekday'] < 4]
test_df = allUserData[allUserData['weekday'] >= 4]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 193
Number of testing data is : 246


In [8]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4, 7]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4, 7]].values
y_test = test_df.iloc[:, 6].values

In [9]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [10]:
clf.score(X_test, y_test)

0.8008130081300813

In [11]:
imp = clf.feature_importances_
imp

array([0.07474598, 0.03433832, 0.23517535, 0.21636921, 0.43937114])

In [19]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.59      0.67      0.62        30
           2       0.85      0.99      0.91        72
           3       1.00      0.82      0.90        33
           4       1.00      0.75      0.86         4
           5       0.00      0.00      0.00         6
           6       0.74      1.00      0.85        14
           8       0.00      0.00      0.00         5
           9       0.50      0.11      0.18         9
          10       0.00      0.00      0.00         3
          11       0.33      0.67      0.44         3
          12       0.00      0.00      0.00         0
          13       0.50      0.40      0.44         5
          16       0.50      0.67      0.57         6
          17       0.93      0.96      0.95        55
          18       0.00      0.00      0.00         1

   micro avg       0.80      0.80      0.80       246
   macro avg       0.46      0.47      0.45       246
weighted avg       0.76   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
print(confusion_matrix(y_test,predictions))

[[20  2  0  0  0  0  0  0  0  3  1  0  3  1  0]
 [ 0 71  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  4 27  0  0  1  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  4  0  0  0  0  0  0  0  1  0  1  0  0  0]
 [ 0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  1  0  1  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  1  0  1  0  0  1  1  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  2  0  0  0  0  0  2  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0  0  4  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  0  0 53  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [21]:
predictions

array([ 2,  2,  3,  2, 17,  1, 17,  1, 17,  2, 17,  1, 17,  2,  2,  2,  2,
        3,  2, 17, 13, 17,  2,  2,  3,  1,  2,  2,  2,  2,  3,  2,  2,  2,
        3,  2,  1,  1, 17,  2, 17,  4, 17,  2,  3,  2, 17,  6,  1,  6, 17,
        2, 17,  4, 17,  9,  2,  2,  3,  2, 17,  6,  1,  6, 17, 13, 17,  2,
        2,  3,  2, 17,  6,  1,  6, 17, 13,  1, 17, 17,  2,  2,  3,  2, 17,
        1,  1, 17,  1,  1, 12,  9, 17,  2,  2,  3, 17,  6, 17,  1,  6, 17,
       17,  4, 17,  2,  2,  3, 17,  6,  2,  2,  3,  1,  2,  2,  2, 13, 11,
       17,  2,  2,  3,  2,  6,  2,  2,  3,  2, 17,  6, 17,  2,  2,  3,  2,
        1,  2,  2,  2,  3, 17, 12,  6,  1, 17,  2,  2,  3,  2, 17,  6,  1,
        6, 17,  2, 11,  2,  2,  3,  2, 17,  6, 11, 17,  2,  2,  3, 16, 16,
       17,  1,  1,  1, 17,  1, 11,  2,  2,  3,  2,  2,  2,  2,  6, 11, 17,
        2,  2,  3, 16,  2, 17,  6,  1,  6, 17,  1, 17,  2,  2,  3, 16, 17,
        1, 17, 17, 17,  2,  2,  3, 16, 17,  1,  1, 17,  1, 17,  1,  2,  2,
        3, 16, 17,  1,  1

In [223]:
location = pd.read_csv("location.csv")
location

,location_id,longitude,latitude,start_time,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
0,1,121.576000,24.986000,NaN,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2,c3,c4,c5
1,2,121.443000,24.433000,NaN,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2,c3,c4,c5
2,3,121.123000,24.765000,NaN,3.677109,NaN,banana,banana1,banana2,banana3,banana4,c1,c2,c3,c4,c5
3,4,121.576839,24.986453,2020/07/16 16:50:30,0.742091,-1.000000,國立政治大學,國立政治大學商學院,萊爾富便利商店,國立政治大學圖書館,國立政治大學商學院圖書分館,university,university,convenience_store,library,library
4,5,121.576855,24.986461,2020/07/16 18:37:11,3432.180398,-1.000000,國立政治大學,國立政治大學商學院,萊爾富便利商店,國立政治大學商學院圖書分館,國立政治大學圖書館,university,university,convenience_store,library,library
5,6,121.581536,25.009585,2020/07/16 19:34:23,0.763942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,121.581536,25.009585,2020/07/16 19:34:23,126.488363,-1.000000,土地公嶺古道,富德公墓辦公室,富德公墓,富德公墓,富德公墓14區,tourist_attraction,point_of_interest,transit_station,transit_station,transit_station
7,8,121.574330,25.027316,2020/07/16 19:36:29,0.912132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,121.574330,25.027316,2020/07/16 19:36:29,1.012450,-1.000000,北星寶宮,石頭公觀音廟,信義公園8號公廁,石頭公觀音亭(法雷窟),舊坡福德宮,place_of_worship,place_of_worship,point_of_interest,point_of_interest,place_of_worship
9,10,121.558394,25.084939,2020/07/16 21:53:54,198.142266,0.079505,維多麗亞酒店,美麗華,開璽吾界,三陽機車 - 鎧詮機車行,美麗華,lodging,transit_station,point_of_interest,car_repair,transit_station


In [224]:
stopLoc = location.drop(['location_id','longitude','latitude','category2','category3','category4','category5'], 1)
stopLoc = stopLoc.reset_index(drop=True)

In [227]:
dropList = []

for row in range(1,len(stopLoc)-1):
    res = set(stopLoc.iloc[row,3:8]) & set(stopLoc.iloc[row-1,3:8])
    
    if len(res) > 1:
        dropList.append(row)
    
    if (stopLoc.iloc[row-1,2] > 3) & (stopLoc.iloc[row+1,2] > 3):
        dropList.append(row)
    
stopLoc = stopLoc.drop(dropList) 

In [228]:
stopLoc = stopLoc.loc[stopLoc['speed'] < 0]
stopLoc = stopLoc.reset_index(drop = True)
stopLoc

,start_time,duration,speed,name1,name2,name3,name4,name5,category1
0,2020/07/16 16:50:30,0.742091,-1.0,國立政治大學,國立政治大學商學院,萊爾富便利商店,國立政治大學圖書館,國立政治大學商學院圖書分館,university
1,2020/07/16 19:34:23,126.488363,-1.0,土地公嶺古道,富德公墓辦公室,富德公墓,富德公墓,富德公墓14區,tourist_attraction
2,2020/07/16 19:36:29,1.012450,-1.0,北星寶宮,石頭公觀音廟,信義公園8號公廁,石頭公觀音亭(法雷窟),舊坡福德宮,place_of_worship
3,2020/07/17 11:53:08,337.173567,-1.0,蜜密牛軋餅,全聯福利中心Pxmart 台北東門,銀翼餐廳,吉野家 東門店,著·咖啡 Draw Coffee 東門店 台北咖啡廳 東門咖啡廳 大安區咖啡廳 台北下午茶 ...,store
4,2020/07/17 11:58:45,18724.143795,-1.0,金華公園,Les Piccola,金華公園地下停車場,Saturn Landing Turkish Coffee 登陸土星土耳其咖啡,O'Steak French Bistro,park
5,2020/07/17 17:10:49,183.142022,-1.0,微風南山,時時香 RICE BAR - 微風南山店,UMAMI 金色三麥,i-Ride TAIPEI 無限飛行事務局,YABI KITCHEN,department_store
6,2020/07/17 17:13:52,2631.127076,-1.0,NIKE Neo19店,全家便利商店 建鑫店,新光三越台北信義新天地A11館,Mo-Mo-Paradise 台北Neo19牧場,馬辣頂級麻辣鴛鴦火鍋-信義店,shoe_store
7,2020/07/17 17:57:43,47.145825,-1.0,溫州鮮餛飩,加州乾洗 Dry Clean & Laundry 自助洗,黑醮食堂,丹頂鶴鋁門窗,饗印印度料理Khanakhazana halal restaurant,restaurant
8,2020/07/17 17:58:30,206.155231,-1.0,台北國際會議中心(TICC),嘟嘟房世貿站,世貿中心(基隆路),Honey pig,黑貓食堂燒肉飯,point_of_interest
9,2020/07/17 18:01:56,391.153719,-1.0,雞二拉麵,客美多咖啡 Komeda's Coffee 敦南信義加盟店,cama café - 台北敦南店,舒服生活 Truffles Living,嘟柏林鐵板燒,restaurant
